In [3]:
#https://stackabuse.com/accessing-the-twitter-api-with-python/
# to save my twitter_credentials.json
import json

## Enter your keys/secrets as strings in the following fields
#credentials = {}  
#credentials['CONSUMER_KEY'] = 'lrPS9OHXHvE035nktii81w5BF'
#credentials['CONSUMER_SECRET'] = 'pF76sVbMXkVPAZZ4eqaDLsQBMeFKCYctvJoPxlamqje8WHq8n7' 
#credentials['ACCESS_TOKEN'] = '757203324970102784-3El1NyAOlq0oulCAzuDfj92Jp0L0fiy'
#credentials['ACCESS_SECRET'] = 'EEpo2ZfpRINSDoh5amp8Jw8Zj985KVRutIwSLKTrOt2lZ'

## Save the credentials object to file
#with open("twitter_credentials.json", "w") as file:  
#    json.dump(credentials, file)

In [4]:
# Import the Twython class
from twython import Twython  
import json

# Load credentials from json file
with open("twitter_credentials.json", "r") as file:  
    creds = json.load(file)

# Instantiate an object
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])



In [46]:
# Create our query
#We'll use only four arguments in the query: 
#q, result_type, count and lang, respectively for the search keyword, type, count, and language of results.
#for more info see https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets.html
query = {'q': 'trump',  
        'result_type': 'popular',#mixed recent popular
        'count': 200,
        'lang': 'en',
        }

In [47]:
import pandas as pd

# Search tweets
dict_ = {'user': [], 'date': [], 'text': [], 'favorite_count': []}  
for status in python_tweets.search(**query)['statuses']:  
    dict_['user'].append(status['user']['screen_name'])
    dict_['date'].append(status['created_at'])
    dict_['text'].append(status['text'])
    dict_['favorite_count'].append(status['favorite_count'])



In [48]:
# Structure data in a pandas DataFrame for easier manipulation
df = pd.DataFrame(dict_)  
df.sort_values(by='favorite_count', inplace=True, ascending=False)  
df.shape 

(15, 4)

# Text Pre-Processing
Depending on your downstream task, cleaning and pre-processing text can involve several different components.  Here are a few important components of Natural Language Processing (NLP) pipelines.

1.  Removing tags: unnecessary content like HTML tags

2.  Removing accented characters: other languages such as French, convert ASCII

3. Removing special characters: adds noise to text, use simple regular expressions (regexes)

4.  Stemming and lemmatization: Stemming remove prefixes and suffixes of word stems (i.e. root words), ex. WATCH is the root stem of WATCHES, WATCHING, and WATCHE.  Lemmatization similar but lexicographically correct word (present in the dictionary).

5.  Expanding contractions: helps text standardization, ex. do not to don’t and I would to I’d

6.  Removing stopwords: Words without meaningful significance (ex. a, an, the, and) but high frequency.

Additional pre-processing: tokenization, removing extra whitespaces,  lower casing and more advanced operations like spelling corrections, grammatical error corrections, removing repeated characters.

In [8]:
import spacy
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
import re
from bs4 import BeautifulSoup
#%run /Users/shahla/Dropbox/MLCourse/Day1/3_sentiment/contractions.py
from contractions import CONTRACTION_MAP #contraction is a  pyhton file in my local computer
import unicodedata

#python3 -m spacy download en #install it in the terminal

nlp = spacy.load('en', parse = False, tag=False, entity=False)
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

In [9]:
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

In [10]:
corpus = ['The sky is blue and beautiful.',
          'Love this blue and beautiful sky!',
          'The quick brown fox jumps over the lazy dog.',
          'The brown fox is quick and the blue dog is lazy!',
          'The sky is very blue and the sky is very beautiful today',
          'The dog is lazy but the brown fox is quick!'    
]
norm_corpus = normalize_corpus(corpus)
norm_corpus

array(['sky blue beautiful', 'love blue beautiful sky',
       'quick brown fox jumps lazy dog', 'brown fox quick blue dog lazy',
       'sky blue sky beautiful today', 'dog lazy brown fox quick'],
      dtype='<U30')

In [11]:
df.text[2]

'We asked @AdamSchiff to see Michael Cohen’s testimony in front of his committee. No answer!\n\nBut Schiff wants the A… https://t.co/fNUkwp6xCV'

In [12]:
normalize_corpus(df.text)

array(['asked adamschiff see michael cohens testimony front committee . answer ! schiff wants … https :// . co / fnukwp6xcv',
       'latest michael cohen request end debate motive keep jail give ne … https :// . co / i99lqosseb',
       'michael cohens attorneys told lawmakers letter cohen discovered substantial files hard dri https :// . co / unyaegcic6',
       'dems first big hearingmichael cohen — ’ going prison lying congress . testifies , lies agai … https :// . co / nbmxnuptww',
       'cnn michael cohen wants stay jail tells congress documents offer assistance investigators',
       'donald trumps day far - michael cohen found old hard drive full evidence trump - capital … https :// . co / zwwqyssuq6',
       'two michael cohen facts 1 sought pardon potus . ( also lied congress , oath ) 2 ) … https :// . co / 55ggph9jc6',
       'cohens attorneys told hill letter thursday cohen discovered substantial files hard drive https :// . co / noaw8lqa9y',
       'michael cohen gift kee

In [13]:
from text_normalizer import normalize_corpus
x=normalize_corpus(df.text)

In [14]:
pd.set_option('display.max_colwidth', -1)
df.text[0][1:100]


'his latest Michael Cohen request should end any debate about his motive. “Keep me out of jail so I '

In [15]:
df.text[0][100:300]

'can give you NE… https://t.co/I99lqoSsEb'

In [25]:
x[9]

'michael cohen attorney lanny davi say ltr hse cmte cohen locate several document believ'

In [19]:
df.text[2]

'We asked @AdamSchiff to see Michael Cohen’s testimony in front of his committee. No answer!\n\nBut Schiff wants the A… https://t.co/fNUkwp6xCV'

In [20]:
df.text[9]

'Two Michael Cohen facts:\n\n1) He sought a pardon from POTUS. (And also lied to Congress about this, under oath)\n2) S… https://t.co/55ggpH9jc6'